In [1]:
%pip install langchain-text-splitters langchain-openai langchain-community langchain-core langchain-cohere chromadb pypdf

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_core.output_parsers import StrOutputParser

from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_cohere import CohereRerank

import os

In [ ]:
os.environ["OPENAI_API_KEY"] = "your_openai_api_key_here"  # Substitua pela sua chave da OpenAI

In [ ]:
os.environ["COHERE_API_KEY"] = "your_cohere_api_key_here"  # Substitua pela sua chave da Cohere

In [6]:
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

llm = ChatOpenAI(model="gpt-4o", max_tokens=600)

In [8]:
pdf_link = "os-sertoes.pdf"

loader = PyPDFLoader(pdf_link, extract_images=False)

pages = loader.load_and_split()

In [9]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=4000,
    chunk_overlap=200,
    length_function=len,
    add_start_index=True
)

chuncks = text_splitter.split_documents(pages)

In [14]:
# Criar vector store
vectordb = Chroma(embedding_function=embeddings)


batch_size = 20  # Processar 20 chunks por vez
total_chunks = len(chuncks)

for i in range(0, total_chunks, batch_size):
    batch = chuncks[i:i + batch_size]
    vectordb.add_documents(batch)
    print(f"  ✓ Processados {min(i + batch_size, total_chunks)}/{total_chunks} chunks")



# Criar retriever "naive" que busca 10 documentos
naive_retriever = vectordb.as_retriever(search_kwargs={"k": 10})



  ✓ Processados 20/658 chunks
  ✓ Processados 40/658 chunks
  ✓ Processados 40/658 chunks
  ✓ Processados 60/658 chunks
  ✓ Processados 60/658 chunks
  ✓ Processados 80/658 chunks
  ✓ Processados 80/658 chunks
  ✓ Processados 100/658 chunks
  ✓ Processados 100/658 chunks
  ✓ Processados 120/658 chunks
  ✓ Processados 120/658 chunks
  ✓ Processados 140/658 chunks
  ✓ Processados 140/658 chunks
  ✓ Processados 160/658 chunks
  ✓ Processados 160/658 chunks
  ✓ Processados 180/658 chunks
  ✓ Processados 180/658 chunks
  ✓ Processados 200/658 chunks
  ✓ Processados 200/658 chunks
  ✓ Processados 220/658 chunks
  ✓ Processados 220/658 chunks
  ✓ Processados 240/658 chunks
  ✓ Processados 240/658 chunks
  ✓ Processados 260/658 chunks
  ✓ Processados 260/658 chunks
  ✓ Processados 280/658 chunks
  ✓ Processados 280/658 chunks
  ✓ Processados 300/658 chunks
  ✓ Processados 300/658 chunks
  ✓ Processados 320/658 chunks
  ✓ Processados 320/658 chunks
  ✓ Processados 340/658 chunks
  ✓ Processados

In [16]:
# Criar o Cohere Rerank compressor
# top_n=3 significa que vai retornar apenas os 3 documentos mais relevantes dos 10 buscados
rerank = CohereRerank(model="rerank-v3.5", top_n=3)

# Criar o Compression Retriever com Rerank
compression_retriever = ContextualCompressionRetriever(
    base_retriever=naive_retriever,  # Retriever que busca 10 documentos
    base_compressor=rerank            # Rerank que seleciona os 3 melhores
)

In [17]:
TEMPLATE = """
Você é um especialista em literatura brasileira. Responda a pergunta abaixo utilizando o contexto informado

Contexto: {context}
    
Pergunta: {question}
"""

prompt = ChatPromptTemplate.from_template(TEMPLATE)

In [19]:
# Função auxiliar para formatar documentos
def format_docs(docs):
    return "\n\n".join([doc.page_content for doc in docs])

# Criar a chain RAG com Compression Retriever (Rerank)
setup_retrieval = RunnableParallel(
    {
        "question": RunnablePassthrough(),
        "context": compression_retriever | format_docs
    }
)

output_parser = StrOutputParser()

# Montar a chain completa
rerank_rag_chain = setup_retrieval | prompt | llm | output_parser

In [3]:
questions = [
    "Qual é a visão de Euclides da Cunha sobre o ambiente natural do sertão nordestino e como ele influencia a vida dos habitantes?",
    "Quais são as principais características da população sertaneja descritas por Euclides da Cunha? Como ele relaciona essas características com o ambiente em que vivem?",
    "Qual foi o contexto histórico e político que levou à Guerra de Canudos, segundo Euclides da Cunha?",
    "Como Euclides da Cunha descreve a figura de Antônio Conselheiro e seu papel na Guerra de Canudos?",
    "Quais são os principais aspectos da crítica social e política presentes em \"Os Sertões\"? Como esses aspectos refletem a visão do autor sobre o Brasil da época?",
]

In [20]:
def answer_question(question):
    return rerank_rag_chain.invoke(question)

In [22]:
# Responder todas as 5 perguntas da lista

for index, question in enumerate(questions, 1):
    try:
        print("=" * 80)
        print(f"PERGUNTA {index}/5:")
        print(question)
        print("=" * 80)

        resposta = answer_question(question)

        print(f"\nRESPOSTA:")
        print(resposta)
        print("\n" + "=" * 80 + "\n")

    except Exception as e:
        print(question)
        print("=" * 80)
        
        resposta = rerank_rag_chain.invoke(question)
        
        print(f"\nRESPOSTA:")
        print(resposta)
        print("\n" + "=" * 80 + "\n")
        
    except Exception as e:
        print(f"❌ Erro ao processar pergunta {index}: {e}\n")
        continue



PERGUNTA 1/5:
Qual é a visão de Euclides da Cunha sobre o ambiente natural do sertão nordestino e como ele influencia a vida dos habitantes?

RESPOSTA:
Euclides da Cunha, em "Os Sertões", descreve o sertão nordestino como um ambiente de extremos e contradições. Ele vê o sertão como um "vale fértil" em momentos de chuva, um grande "pomar sem dono", mas que se transforma drasticamente com a chegada do período de seca, assumindo uma atmosfera "asfixiante" e um solo "empedrado" que impõe um "espasmo assombrador". Essa visão enfatiza a natureza antitética da região, com ciclos de abundância e escassez.

Para Euclides, a seca não é apenas uma questão de falta d'água a ser resolvida por meios locais, como cisternas ou poços artesianos. Ele entende que o verdadeiro desafio é o combate a um "deserto" mais amplo, que afeta profundamente a "economia geral da Vida". Assim, a seca é vista como uma manifestação do "martírio secular da Terra", o que reflete um ciclo mais amplo de sofrimento humano e 